# aQa Final Project: Qualitive comparison of Quantum Neural Networks and Classical Neural Networks

Firstly, import the correct libraries. This may take some time.

In [ ]:
import tensorflow as tf

import cirq
import numpy as np
import seaborn as sns
import time

from scipy.optimize import minimize

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

%matplotlib inline
import matplotlib.pyplot as plt

Next, we will define the depth and the number of qubits. We can sweep these variables later. <br>
We can also generate the rotations that will generate the random data.

In [ ]:
np.random.seed(18)
depth = 3
n_qubits = 3

randomRotations = np.random.uniform(-2*np.pi , 2 * np.pi, (depth * n_qubits))

# Circuits

### Quantum Neural network

We first define the quantum neural network. This is a simple circuit with a few layers of rotations and entanglements. We will use this circuit to classify the data.

In [ ]:
def buildQuantumModel(encoding: np.array, parameters: np.array, depth : int, n_qubits : int) -> None:
    qubits = cirq.LineQubit.range(n_qubits)

    #encoding layer
    yield [cirq.ry(encoding[i])(qubits[i]) for i in range(n_qubits)]
    yield [cirq.rx(encoding[i])(qubits[i]) for i in range(n_qubits)]

    #variational layer
    for l in range(depth):
        for i in range(n_qubits):
            yield cirq.rx(parameters[i + l*n_qubits])(qubits[i])
        for i in range(n_qubits-1):
            yield cirq.CZ(qubits[i], qubits[i+1])


    #readout layer
    yield cirq.measure(qubits[0], key='z0')

simulator = cirq.Simulator()

Secondly, we define the `partial quantum model`, which has the same structure as the quantum neural network, but with an extra encoding layer inbetween even layers.

In [ ]:
def buildPartialQuantumModel(encoding: np.array, parameters: np.array, depth : int, n_qubits : int) -> None:
    qubits = cirq.LineQubit.range(n_qubits)

    #encoding layer
    yield [cirq.ry(encoding[i])(qubits[i]) for i in range(n_qubits)]
    yield [cirq.rx(encoding[i])(qubits[i]) for i in range(n_qubits)]



    #variational layer
    for l in range(depth):
        for i in range(n_qubits):
            yield cirq.rx(parameters[i + l*n_qubits])(qubits[i])
        for i in range(n_qubits-1):
            yield cirq.CZ(qubits[i], qubits[i+1])

        if l % 2 == 0:
            yield [cirq.ry(encoding[i])(qubits[i]) for i in range(n_qubits)]
            yield [cirq.rx(encoding[i])(qubits[i]) for i in range(n_qubits)]


    #readout layer
    yield cirq.measure(qubits[0], key='z0')

simulator = cirq.Simulator()

Thirdly, we define the `reuploading quantum model`, which has the same structure as the quantum neural network, but with an extra encoding layer inbetween every layer.

In [ ]:
def buildReuploadQuantumModel(encoding: np.array, parameters: np.array, depth : int, n_qubits : int) -> None:
    qubits = cirq.LineQubit.range(n_qubits)

    
    #encoding layer
    yield [cirq.ry(encoding[i])(qubits[i]) for i in range(n_qubits)]
    yield [cirq.rx(encoding[i])(qubits[i]) for i in range(n_qubits)]



    #variational layer
    for l in range(depth):
        for i in range(n_qubits):
            yield cirq.rx(parameters[i + l*n_qubits])(qubits[i])
        for i in range(n_qubits-1):
            yield cirq.CZ(qubits[i], qubits[i+1])

        yield [cirq.ry(encoding[i])(qubits[i]) for i in range(n_qubits)]
        yield [cirq.rx(encoding[i])(qubits[i]) for i in range(n_qubits)]


    #readout layer
    yield cirq.measure(qubits[0], key='z0')

simulator = cirq.Simulator()

As the quantum neural network output needs to be simulated, we need to built these for each of the QNN.

In [ ]:
def runCircuit(data : np.array, parameters : np.array) -> float:


    quantumModel = cirq.Circuit(buildQuantumModel(data, parameters, depth, n_qubits))
    result = simulator.run(quantumModel, repetitions=100)

    result = np.mean(result.measurements['z0'])

    return result

def runPartialCircuit(data : np.array, parameters : np.array) -> float:

    
    quantumModel = cirq.Circuit(buildPartialQuantumModel(data, parameters, depth, n_qubits))
    result = simulator.run(quantumModel, repetitions=100)
    
    result = np.mean(result.measurements['z0'])
    
    return result

def runReuploadCircuit(data : np.array, parameters : np.array) -> float:

    
    quantumModel = cirq.Circuit(buildReuploadQuantumModel(data, parameters, depth, n_qubits))
    result = simulator.run(quantumModel, repetitions=100)
    
    result = np.mean(result.measurements['z0'])
    
    return result
      


### Classical Neural network

Lastly, we define the classical neural network. This is a simple neural network with a simple leaky relu dense layers and a sigmoid output layer.

In [ ]:
def buildClassicalModel(depth : int , trainable : bool) -> tf.keras.Sequential:

    classicModel = tf.keras.Sequential()
    
    for l in range(depth):
        classicModel.add(tf.keras.layers.Dense(n_qubits, activation='leaky_relu', trainable=trainable))

    classicModel.add(tf.keras.layers.Dense(1, activation='sigmoid', trainable=trainable))

    return classicModel

classicModel = buildClassicalModel(depth, True)
dataClassicalModel = buildClassicalModel(depth, False)

## Data generation

### Classic data

Firstly, we sample a define a sampler, creating the 3-dimensional dataset.

In [ ]:
def generateClassicSamples(samples : int) -> np.array:

    empty = np.array([])

    for i in range(samples):
        empty = np.append(empty, np.random.uniform(-1, 1, 2))

    empty = empty.reshape(samples, 2)
    

    data = np.zeros((empty.shape[0], empty.shape[1] + 1))
    data[:,:2] = empty
    data[:,-1] = empty[:,0]*empty[:,1]

    return data

Now, we classify 1000 of these samples using the fixed classical neural network.

In [ ]:
dataClassicalModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classicData = generateClassicSamples(1000)


classicLabels = dataClassicalModel.predict(classicData)
onelabel = zerolabel = 0

for i in range(len(classicLabels)):
    if classicLabels[i] > 0.498:
        classicLabels[i] = 1
        onelabel += 1
    else:
        classicLabels[i] = 0
        zerolabel += 1

The split between the data can be shown in the 2d-space.

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Classic Data')
ax.set_ylabel('x2')
ax.set_xlabel('x1')
sns.scatterplot(x=classicData[:,0], y=classicData[:,1], hue=classicLabels.flatten(), ax=ax)
fig.savefig('figures/classicData.png')

## Training & Testing

Firstly, we test on the classical model. Lastly, we initialize the COBYLA optimizer and train the three quantum neural networks.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(classicData, classicLabels, test_size=0.33, random_state=4)

classicModel = buildClassicalModel(depth, True)
classicModel.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])

y_train = np.array(y_train)
y_test = np.array(y_test)


start = time.time()
classicModel.fit(X_train, y_train, epochs=50)
end = time.time()
print(classicModel.evaluate(X_test, y_test))
print("Time: ", end - start)

In [ ]:
def calculateLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)
    

cost = []


def optimize(parameters : np.array) -> float:

    singleCost = calculateLoss(parameters, X_train, y_train)
    cost.append(singleCost)

    return singleCost


start = time.time()
result = minimize(optimize, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculatePartialLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runPartialCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizePartial(parameters : np.array) -> float:
    
        singleCost = calculatePartialLoss(parameters, X_train, y_train)
        cost.append(singleCost)
    
        return singleCost


start = time.time()
result = minimize(optimizePartial, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runPartialCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculateReuploadLoss(parameters : np.array, data: np.array, labels: np.array) -> float:


    probs = [runReuploadCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizeReupload(parameters : np.array) -> float:
        
        singleCost = calculateReuploadLoss(parameters, X_train, y_train)
        cost.append(singleCost)
        
        return singleCost



start = time.time()
result = minimize(optimizeReupload, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runReuploadCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

### Quantum data

We apply the same process as with the classical data, only now sampling between 0 and 2*pi

In [ ]:
def generateQuantumSamples(samples : int) -> np.array:

    empty = np.array([])

    for i in range(samples):
        empty = np.append(empty, np.random.uniform(0, 2*np.pi, 2))

    empty = empty.reshape(samples, 2)
    

    data = np.zeros((empty.shape[0], empty.shape[1] + 1))
    data[:,:2] = empty
    data[:,-1] = empty[:,0]*empty[:,1]

    return data

In [ ]:
quantumData = generateQuantumSamples(1000)
quantumLabels = [runCircuit(quantumData[i,:], randomRotations) for i in range(len(quantumData))]

onelabel = zerolabel = 0

for i in range(len(quantumLabels)):
    if quantumLabels[i] > 0.5:
        quantumLabels[i] = 1
        onelabel += 1
    else:
        quantumLabels[i] = 0
        zerolabel += 1


In [ ]:
fig, ax = plt.subplots()
ax.set_title('Basic Quantum Neural Network Generated Data')
ax.set_ylabel('x2')
ax.set_xlabel('x1')
sns.scatterplot(x=quantumData[:,0], y=quantumData[:,1], hue=quantumLabels, ax=ax)
fig.savefig('figures/quantumData.png')

In [ ]:
partialQuantumData = generateQuantumSamples(1000)
partialQuantumLabels = [runPartialCircuit(quantumData[i,:], randomRotations) for i in range(len(quantumData))]

onelabel = zerolabel = 0

for i in range(len(partialQuantumLabels)):
    if partialQuantumLabels[i] > 0.5:
        partialQuantumLabels[i] = 1
        onelabel += 1
    else:
        partialQuantumLabels[i] = 0
        zerolabel += 1


In [ ]:
fig, ax = plt.subplots()
ax.set_title('Partial Quantum Neural Network Generated Data')
ax.set_ylabel('x2')
ax.set_xlabel('x1')
sns.scatterplot(x=partialQuantumData[:,0], y=partialQuantumData[:,1], hue=partialQuantumLabels, ax=ax)
fig.savefig('figures/partialQuantumData.png')

In [ ]:
reuploadQuantumData = generateQuantumSamples(1000)
reuploadQuantumLabels = [runReuploadCircuit(quantumData[i,:], randomRotations) for i in range(len(quantumData))]

onelabel = zerolabel = 0

for i in range(len(reuploadQuantumLabels)):
    if reuploadQuantumLabels[i] > 0.5:
        reuploadQuantumLabels[i] = 1
        onelabel += 1
    else:
        reuploadQuantumLabels[i] = 0
        zerolabel += 1


In [ ]:
fig, ax = plt.subplots()
ax.set_title('Reuploading Quantum Neural Network Generated Data')
ax.set_ylabel('x2')
ax.set_xlabel('x1')
sns.scatterplot(x=reuploadQuantumData[:,0], y=reuploadQuantumData[:,1], hue=reuploadQuantumLabels, ax=ax)
fig.savefig('figures/reuploadQuantumData.png')

## Training & Testing

Same as the training and testing on the classical dataset, now for each of the datasets generated by the quantum neural networks.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(quantumData, quantumLabels, test_size=0.33, random_state=4)

classicModel = buildClassicalModel(depth, True)
classicModel.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])

y_train = np.array(y_train)
y_test = np.array(y_test)


start = time.time()
classicModel.fit(X_train, y_train, epochs=50)
end = time.time()
print(classicModel.evaluate(X_test, y_test))
print("Time: ", end - start)

In [ ]:
def calculateLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    loss = 0

    probs = [runCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)
    

cost = []


def optimize(parameters : np.array) -> float:

    singleCost = calculateLoss(parameters, X_train, y_train)
    cost.append(singleCost)

    return singleCost


start = time.time()
result = minimize(optimize, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculatePartialLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runPartialCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizePartial(parameters : np.array) -> float:
    
        singleCost = calculatePartialLoss(parameters, X_train, y_train)
        cost.append(singleCost)
    
        return singleCost


start = time.time()
result = minimize(optimizePartial, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runPartialCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculateReuploadLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runReuploadCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizeReupload(parameters : np.array) -> float:
        
        singleCost = calculateReuploadLoss(parameters, X_train, y_train)
        cost.append(singleCost)
        
        return singleCost



start = time.time()
result = minimize(optimizeReupload, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runReuploadCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(partialQuantumData, partialQuantumLabels, test_size=0.33, random_state=4)

classicModel = buildClassicalModel(depth, True)
classicModel.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])

y_train = np.array(y_train)
y_test = np.array(y_test)


start = time.time()
classicModel.fit(X_train, y_train, epochs=50)
end = time.time()
print(classicModel.evaluate(X_test, y_test))
print("Time: ", end - start)

In [ ]:
def calculateLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)
    

cost = []


def optimize(parameters : np.array) -> float:

    singleCost = calculateLoss(parameters, X_train, y_train)
    cost.append(singleCost)

    return singleCost


start = time.time()
result = minimize(optimize, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculatePartialLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runPartialCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizePartial(parameters : np.array) -> float:
    
        singleCost = calculatePartialLoss(parameters, X_train, y_train)
        cost.append(singleCost)
    
        return singleCost


start = time.time()
result = minimize(optimizePartial, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runPartialCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculateReuploadLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runReuploadCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizeReupload(parameters : np.array) -> float:
        
        singleCost = calculateReuploadLoss(parameters, X_train, y_train)
        cost.append(singleCost)
        
        return singleCost



start = time.time()
result = minimize(optimizeReupload, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runReuploadCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reuploadQuantumData, reuploadQuantumLabels, test_size=0.33, random_state=4)

classicModel = buildClassicalModel(depth, True)
classicModel.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])

y_train = np.array(y_train)
y_test = np.array(y_test)


start = time.time()
classicModel.fit(X_train, y_train, epochs=50)
end = time.time()
print(classicModel.evaluate(X_test, y_test))
print("Time: ", end - start)

In [ ]:
def calculateLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    probs = [runCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)
    

cost = []


def optimize(parameters : np.array) -> float:

    singleCost = calculateLoss(parameters, X_train, y_train)
    cost.append(singleCost)

    return singleCost


start = time.time()
result = minimize(optimize, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculatePartialLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    loss = 0

    probs = [runPartialCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizePartial(parameters : np.array) -> float:
    
        singleCost = calculatePartialLoss(parameters, X_train, y_train)
        cost.append(singleCost)
    
        return singleCost


start = time.time()
result = minimize(optimizePartial, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runPartialCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)

In [ ]:
def calculateReuploadLoss(parameters : np.array, data: np.array, labels: np.array) -> float:

    loss = 0

    probs = [runReuploadCircuit(data[i,:], parameters) for i in range(len(data))]

    for i in range(len(probs)):
        if probs[i] > 0.5:
            probs[i] = 1
        else:
            probs[i] = 0

    return mse(labels, probs)


cost = []


def optimizeReupload(parameters : np.array) -> float:
        
        singleCost = calculateReuploadLoss(parameters, X_train, y_train)
        cost.append(singleCost)
        
        return singleCost



start = time.time()
result = minimize(optimizeReupload, randomRotations, method='COBYLA', options={"maxiter": 50})
predictedLabels = [runReuploadCircuit(X_test[i,:], result.x) for i in range(len(X_test))]
predictedLabels = [1 if predictedLabels[i] > 0.5 else 0 for i in range(len(predictedLabels))]
end = time.time()
print("MSE: ", str(mse(y_test, predictedLabels)))
print("Time", end - start)